### Step 1 - Read the Information from the Blobs

In this step, we will structure the data retrieved from Azure Document Intelligence. The data will be outputted as a JSON file, and it is our role to process and organize it. Some of the data will be structured into tables, while other data will be formatted as text. This step ensures that the extracted information is organized in a meaningful way for further analysis and usage.

In [42]:
import os
import json
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

def read_json_files_from_blob(folder_path):
    # Retrieve the connection string from the environment variables
    connection_string = os.getenv('connection_string')

    # Ensure the connection string is not None
    if connection_string is None:
        raise ValueError("The connection string environment variable is not set.")

    # Create a BlobServiceClient
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)

    # Get the container client
    container_client = blob_service_client.get_container_client("bankdetail")

    # List all blobs in the specified folder
    blob_list = container_client.list_blobs(name_starts_with=folder_path)

    # Filter out JSON files and read their contents
    for blob in blob_list:
        if blob.name.endswith('.json'):
            blob_client = container_client.get_blob_client(blob.name)
            blob_data = blob_client.download_blob().readall()
            data = json.loads(blob_data)
            # print(f"Contents of {blob.name}:")
            # print(json.dumps(data, indent=2))
            # print("\n")
            return data 

#### Loan Agreements

In [43]:
loanagreement = read_json_files_from_blob("loanagreements")

#### Loan Forms

In [44]:
loanform = read_json_files_from_blob("loanform")

#### Pay Stubs

In [100]:
paystub = read_json_files_from_blob("paystubs")

### Step 3 - Data Structuring


In this step, you will read JSON data from Azure Blob Storage, clean the data to retain only the text content, and remove any unnecessary formatting such as newlines and spaces. Follow the instructions below to complete this step.

#### Loan Agreements

In [46]:
def clean_json_data(json_data):
    # Extract relevant text content from the JSON
    content = []

    # Extract text from paragraphs
    paragraphs = json_data.get("paragraphs", [])
    for paragraph in paragraphs:
        content.append(paragraph.get("text", "").strip())

    # Extract text from pages and lines
    pages = json_data.get("pages", [])
    for page in pages:
        for line in page.get("lines", []):
            content.append(line.get("text", "").strip())

    # Join all text content into a single string with spaces between components
    plain_text_content = " ".join(content)

    return plain_text_content

# Clean the JSON data
cleaned_data = clean_json_data(loanagreement)

# Print the cleaned data
print(json.dumps(cleaned_data, indent=2))

"LOAN AGREEMENT This Loan Agreement (\"Agreement\") is made and entered into on August 1, 2024, by and between: \u00b7 Lender: Contoso Bank Address: 123 Finance Avenue, Madison, WI 53703 Contact Number: (555) 123-4567 Email: lending@contosobank.com . Borrower: Jane Elizabeth Smith Address: 456 Oak Avenue, Unit 10, Madison, WI 53703 Contact Number: (555) 234-5678 Email: jane.smith90@example.com 1. Loan Amount and Purpose 1.1 Loan Amount: The Lender agrees to loan the Borrower the principal sum of $30,000.00 (thirty thousand dollars), referred to as the \"Loan.\" 1.2 Purpose of Loan: The Loan shall be used exclusively for the purchase of a vehicle, specifically a 2022 Toyota Camry. 2. Interest Rate 2.1 Interest Rate: The Loan shall bear interest at an annual fixed rate of 5.5%. 2.2 Accrual: Interest shall begin to accrue on the Loan from the date the funds are disbursed to the Borrower. 3. Loan Term 3.1 Term: The term of this Loan shall be 5 years (60 months), commencing on August 1, 202

In [47]:
# Clean the JSON data
cleaned_data = clean_json_data(loanagreement)

# Print the cleaned data
print(json.dumps(cleaned_data, indent=2))

"LOAN AGREEMENT This Loan Agreement (\"Agreement\") is made and entered into on August 1, 2024, by and between: \u00b7 Lender: Contoso Bank Address: 123 Finance Avenue, Madison, WI 53703 Contact Number: (555) 123-4567 Email: lending@contosobank.com . Borrower: Jane Elizabeth Smith Address: 456 Oak Avenue, Unit 10, Madison, WI 53703 Contact Number: (555) 234-5678 Email: jane.smith90@example.com 1. Loan Amount and Purpose 1.1 Loan Amount: The Lender agrees to loan the Borrower the principal sum of $30,000.00 (thirty thousand dollars), referred to as the \"Loan.\" 1.2 Purpose of Loan: The Loan shall be used exclusively for the purchase of a vehicle, specifically a 2022 Toyota Camry. 2. Interest Rate 2.1 Interest Rate: The Loan shall bear interest at an annual fixed rate of 5.5%. 2.2 Accrual: Interest shall begin to accrue on the Loan from the date the funds are disbursed to the Borrower. 3. Loan Term 3.1 Term: The term of this Loan shall be 5 years (60 months), commencing on August 1, 202

#### Loan Form

In [48]:
def clean_json_data_with_tables(json_data):
    def combine_table_rows(table):
        # Dictionary to store the combined content for row_index 1 and 2
        combined_content = {}

        # Iterate over the cells to combine content for row 1 and row 2
        for cell in table['cells']:
            row_index = cell['row_index']
            col_index = cell['column_index']
            content = cell['content'].strip()  # Strip any leading/trailing whitespace

            if row_index == 1:
                # Start with the content in row 1
                combined_content[col_index] = content
            elif row_index == 2:
                # Append the content in row 2 to the corresponding column in row 1
                if col_index in combined_content:
                    combined_content[col_index] += " " + content
                    print(combined_content)
                else:
                    combined_content[col_index] = content

        # Create a new list for the modified cells
        new_cells = []

        # Add the original row 0 cells to the new list
        for cell in table['cells']:
            if cell['row_index'] == 0:
                new_cells.append(cell)

        # Create new cells for the combined row (row_index 1)
        for col_index in range(table['column_count']):
            combined_cell = {
                "row_index": 1,
                "column_index": col_index,
                "content": combined_content.get(col_index, ""),  # Get the combined content or an empty string
                "bounding_regions": []  # Leave bounding regions empty for this example
            }
            new_cells.append(combined_cell)

        # Return the modified table with only two rows
        return {
            "row_count": 2,  # Updated row count
            "column_count": table['column_count'],
            "bounding_regions": table['bounding_regions'],  # Keep the original bounding regions
            "cells": new_cells
        }

    def convert_table_data_to_json(table_data):
        # Initialize the resulting table structure
        table = {
            "Employer Name": [],
            "Position": [],
            "Employment Duration": [],
            "Monthly Income": [],
            "Contact Number": []
        }

        # Iterate through the cells in the table data
        for cell in table_data['cells']:
            row_index = cell['row_index']
            col_index = cell['column_index']
            content = cell['content'].strip()  # Remove any leading/trailing whitespace

            if row_index == 1:
                if col_index == 0:
                    table["Employer Name"].append(content)
                elif col_index == 1:
                    table["Position"].append(content)
                elif col_index == 2:
                    table["Employment Duration"].append(content)
                elif col_index == 3:
                    table["Monthly Income"].append(content)
                elif col_index == 4:
                    table["Contact Number"].append(content)

        return table

    # Extract relevant text content from the JSON
    content = []

    # Extract text from paragraphs
    paragraphs = json_data.get("paragraphs", [])
    for paragraph in paragraphs:
        content.append(paragraph.get("text", "").strip())

    # Extract text from pages and lines
    pages = json_data.get("pages", [])
    for page in pages:
        for line in page.get("lines", []):
            content.append(line.get("text", "").strip())

        # Extract text from tables
        tables = page.get("tables", [])
        for table in tables:
            # Combine rows 1 and 2 in each column
            combined_table = combine_table_rows(table)
            table_content = convert_table_data_to_json(combined_table)
            # You can append the JSON structure or its string representation to the content
            content.append(str(table_content))

    # Join all text content into a single string with spaces between components
    plain_text_content = " ".join(content)

    return plain_text_content

In [49]:
pre_cleaned_data = clean_json_data_with_tables(loanform)

In [50]:
import re

def clean_loan_application_file(text):
 
    cleaned_data = {}

    # Extract the category from the first three words
    category_match = re.search(r'(\w+\s+\w+\s+\w+)', text)
    if category_match:
        cleaned_data['Category'] = category_match.group(1)
    
    # Extract Applicant Information
    applicant_info = re.search(r'Applicant Information(.*?)Employment and Income Details', text, re.DOTALL)
    if applicant_info:
        applicant_info_text = applicant_info.group(1)
        cleaned_data['Applicant Information'] = {
            'Full Name': re.search(r'Full Name:\s*(.*?)Date of Birth:', applicant_info_text, re.DOTALL).group(1).strip(),
            'Date of Birth': re.search(r'Date of Birth:\s*(.*?)Social Security Number:', applicant_info_text, re.DOTALL).group(1).strip(),
            'Social Security Number': re.search(r'Social Security Number:\s*(.*?)Contact Number:', applicant_info_text, re.DOTALL).group(1).strip(),
            'Contact Number': re.search(r'Contact Number:\s*(.*?)Email Address:', applicant_info_text, re.DOTALL).group(1).strip(),
            'Email Address': re.search(r'Email Address:\s*(.*?)Physical Address:', applicant_info_text, re.DOTALL).group(1).strip(),
            'Physical Address': re.search(r'Physical Address:\s*(.*)', applicant_info_text, re.DOTALL).group(1).strip(),
        }

    # Extract Employment and Income Details
    employment_info = re.search(r'Employment and Income Details(.*?)Loan Information', text, re.DOTALL)
    if employment_info:
        employment_info_text = employment_info.group(1)
        cleaned_data['Employment and Income Details'] = {
            'Employer Name': re.search(r'Employer Name\s*(.*?)Position', employment_info_text, re.DOTALL).group(1).strip(),
            'Position': re.search(r'Position\s*(.*?)Employment Duration', employment_info_text, re.DOTALL).group(1).strip(),
            'Employment Duration': re.search(r'Employment Duration\s*(.*?)Monthly Income', employment_info_text, re.DOTALL).group(1).strip(),
            'Monthly Income': re.search(r'Monthly Income\s*\$?(.*?)Contact Number', employment_info_text, re.DOTALL).group(1).strip(),
            'Employer Contact Number': re.search(r'Contact Number\s*(.*)', employment_info_text, re.DOTALL).group(1).strip(),
        }

    # Extract Loan Information
    loan_info = re.search(r'Loan Information(.*)', text, re.DOTALL)
    if loan_info:
        loan_info_text = loan_info.group(1)
        cleaned_data['Loan Information'] = {
            'Loan Amount Requested': re.search(r'Loan Amount Requested:\s*\$?(.*?)Purpose of Loan:', loan_info_text, re.DOTALL).group(1).strip(),
            'Purpose of Loan': re.search(r'Purpose of Loan:\s*(.*?)Loan Term Desired:', loan_info_text, re.DOTALL).group(1).strip(),
            'Loan Term Desired': re.search(r'Loan Term Desired:\s*(.*)', loan_info_text, re.DOTALL).group(1).strip(),
        }

    return cleaned_data

# Example usage
cleaned_data = clean_loan_application_file(pre_cleaned_data)
print(json.dumps(cleaned_data, indent=2))

{
  "Category": "Loan Application Form",
  "Applicant Information": {
    "Full Name": "Jane Elizabeth Smith",
    "Date of Birth": "08/22/1990",
    "Social Security Number": "987-65-4321",
    "Contact Number": "(555) 234-5678",
    "Email Address": "jane.smith90@example.com",
    "Physical Address": "456 Oak Avenue, Unit 10, Madison, WI 53703"
  },
  "Employment and Income Details": {
    "Employer Name": "",
    "Position": "",
    "Employment Duration": "",
    "Monthly Income": "",
    "Employer Contact Number": "Horizon Store Manager 3 years $4,583.33 (555) 789- Retailers 2345"
  },
  "Loan Information": {
    "Loan Amount Requested": "30,000",
    "Purpose of Loan": "Vehicle Purchase",
    "Loan Term Desired": "5 years Applicant's Signature:"
  }
}


#### Paystubs

In [115]:
def clean_form_recognizer_result(data):
    for page in data.get("pages", []):
        for line in page.get("lines", []):
            # Check if the line contains the word "table"
            if "table" in line.get("text", "").lower():
                continue  # Keep everything if "table" is in the text
            # Keep only the "text" key
            line_keys = list(line.keys())
            for key in line_keys:
                if key != "text":
                    del line[key]
    
    # Create structured tables
    structured_tables = create_structured_tables(data.get("tables", []))
    data["structured_tables"] = structured_tables
    
    return data

def create_structured_tables(tables):
    structured_tables = []
    for table in tables:
        row_count = table.get("row_count", 0)
        column_count = table.get("column_count", 0)
        cells = table.get("cells", [])
        
        # Initialize an empty table
        structured_table = [["" for _ in range(column_count)] for _ in range(row_count)]
        
        # Populate the table with cell content
        for cell in cells:
            row_index = cell.get("row_index", 0)
            column_index = cell.get("column_index", 0)
            content = cell.get("content", "")
            structured_table[row_index][column_index] = content
        
        structured_tables.append(structured_table)
    
    return structured_tables

import pandas as pd
def tables_to_dataframes(structured_tables):
    dataframes = []
    for table in structured_tables:
        df = pd.DataFrame(table)
        dataframes.append(df)
    return dataframes

cleaned_data = clean_form_recognizer_result(paystub)
dataframes = tables_to_dataframes(cleaned_data["structured_tables"])

df_list = []

for df in dataframes:
    print(df)
    df_list.append(df)

combined_df = pd.concat(df_list, ignore_index=True)
combined_df.to_excel('dataframes_combined.xlsx', index=False)

cleaned_data = clean_form_recognizer_result(paystub)
print(cleaned_data)

                0             1       2                 3  \
0     Description  Hours Worked    Rate  Current Earnings   
1     Regular Pay           160  $28.65         $4,583.33   
2    Overtime Pay             5  $42.98           $214.90   
3           Bonus           N/A     N/A           $250.00   
4  Total Earnings                               $5,048.23   

                       4  
0  Year-to-Date Earnings  
1             $32,083.31  
2              $1,289.40  
3              $1,750.00  
4             $35,122.71  
                     0               1                    2
0          Description  Current Amount  Year-to-Date Amount
1          Federal Tax         $800.00            $5,600.00
2            State Tax         $200.00            $1,400.00
3      Social Security         $314.99            $2,204.93
4             Medicare          $73.66              $515.49
5  401(k) Contribution         $250.00            $1,750.00
6     Health Insurance         $150.00            $